In [ ]:
import openpyxl
import operator
import json


def load_requirements_from_json(filepath):
    """Loads requirements from a JSON file."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            requirements_data = json.load(f)
        return requirements_data
    except FileNotFoundError:
        print(f"Error: Requirements file not found at '{filepath}'.")
        return None  # Or raise an exception if you prefer
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in '{filepath}'.")
        return None  # Or raise an exception


def course_parses(file_path):
    # Load the Excel workbook
    workbook = openpyxl.load_workbook(file_path)

    # Choose the active sheet
    sheet = workbook.active

    # Initialize an empty dictionary to store courses
    courses = {}

    def varInitializer():
        course_code = None
        course_name = None
        schedule = None
        credits = None

        return course_code, course_name, schedule, credits

    course_code, course_name, schedule, credits = varInitializer()

    # Loop through all rows
    for row in sheet.iter_rows(min_row=1,
                               max_row=sheet.max_row,
                               min_col=1,
                               max_col=sheet.max_column):
        for cell in row:
            cell_value = cell.value

            # Get the background color
            if cell.fill.patternType == "solid":
                background_color = cell.fill.start_color.rgb  # RGB format (e.g., 'FFFFFF00')
            else:
                background_color = None

            if cell_value:
                course_code, course_name, schedule, credits = cell_value.splitlines()

                # Process the schedules into a list of dictionaries
                schedule_list = []
                for time_slots in schedule.strip("[]").split(";"):
                    parts = time_slots.strip().split(";")
                    for part in parts:
                        day, interval = part.split(" ")
                        schedule_list.append({
                            "day"     : day,
                            "interval": interval})

                courses[course_code] = {

                    "course_code": course_code,
                    "course_name": course_name.strip("()"),
                    "credits"    : credits,
                    "schedule"   : schedule_list,
                }
            course_code, course_name, schedule, credits = varInitializer()
    return courses


courses = course_parses("../data/Courses.xlsx")
requirements = load_requirements_from_json("../data/requirements.json")

In [ ]:
min_credit = 34
max_credit = 43
possible_programs = []


def check_satisfied(needed, count):
    # Define a dictionary to map operators to their corresponding functions
    condition_operators = {
        "=" : operator.eq,
        "<=": operator.le,
        ">=": operator.ge,
        "<" : operator.lt,
        ">" : operator.gt
    }

    # Extract the operator and value
    for op in condition_operators.keys():
        if needed.startswith(op):
            value = int(
                needed[len(op):])  # Extract the number after the operator
            condition_func = condition_operators[op]
            return condition_func(count, value)
    raise ValueError(f"Invalid condition: {needed}")


# Helper function to convert a time string to a datetime object (for comparison)
def time_to_minutes(time_str) -> int:
    """Convert time in 'HH.MM' format to total minutes"""
    hours, minutes = map(int, time_str.split("."))
    return hours * 60 + minutes


# Helper function to convert a time string to a datetime object (for comparison)
def minutes_to_time(minutes_int) -> str:
    hours, minutes = divmod(minutes_int, 60)
    hours = str(hours)
    minutes = str(minutes)
    if len(hours) == 1:
        hours = "0" + hours
    if len(minutes) == 1:
        minutes = "0" + minutes
    return hours + '.' + minutes


# Helper function to parse the schedule string
def parse_time_slot(time_slot_str):
    """Parse a schedule string and return a list of (day, start_time, end_time) tuples"""
    day = time_slot_str["day"]
    start_time, end_time = time_slot_str["interval"].split("-")
    return day, time_to_minutes(start_time), time_to_minutes(end_time)


# Helper function to check if two time slots overlap
def check_program_course_conflict(current_program_param, candidate_schedule_param):
    for current_program_time_slot in current_program_param["schedule"]:
        for candidate_schedule_time_slot in candidate_schedule_param:

            compare1 = parse_time_slot(candidate_schedule_time_slot)
            compare2 = parse_time_slot(current_program_time_slot)
            if compare1[0] == compare2[0]:
                if compare1[1] < compare2[2] and compare1[2] > compare2[1]:
                    return True  # Conflict
    return False  # No conflict


def calculate_program_stats(program):
    """Calculates total days and total hours for a program."""
    days = set()
    total_minutes = 0
    for course_code in program['courses']:
        for schedule_entry in courses[course_code]['schedule']:
            days.add(schedule_entry['day'])
            start_minutes = time_to_minutes(schedule_entry['interval'].split('-')[0])
            end_minutes = time_to_minutes(schedule_entry['interval'].split('-')[1])
            total_minutes += (end_minutes - start_minutes)

    program["total_days"] = len(days)
    program["total_hours"] = total_minutes / 60.0  # Convert minutes to hours
    return program


def is_program_valid(program):
    total_credits = sum(int(courses[course_code]['credits']) for course_code in program['courses'])
    if not (min_credit <= total_credits <= max_credit):
        return False

    for req in requirements:
        count = 0
        for course_code in program['courses']:
            if course_code in req['candidates']:
                count += 1
        if not check_satisfied(req['needed'], count):
            return False

    program["total_credit"] = total_credits  # Correctly calculate total_credit here
    program = calculate_program_stats(program)  # Calculate total_days and total_hours
    return True


def generate_programs(requirement_index, current_program_courses):
    if requirement_index == len(requirements):
        program = {
            "courses" : current_program_courses,
            "schedule": []
        }
        for course_code in current_program_courses:
            program["schedule"].extend(courses[course_code]["schedule"])

        if is_program_valid(program):
            #print(f"Found valid program: {program}")
            possible_programs.append(program)
        return

    # Option 1: Try to fulfill the requirement with candidates
    current_requirement = requirements[requirement_index]
    course_options = current_requirement["candidates"]
    needed_condition = current_requirement["needed"]

    # Determine how many courses we *can* take for this requirement (based on 'needed')
    max_courses_for_req = float('inf')  # default max is infinity
    if "<=" in needed_condition:
        max_courses_for_req = int(needed_condition.split("<=")[1])
    elif "=" in needed_condition:
        max_courses_for_req = int(needed_condition.split("=")[1])
    elif "<" in needed_condition:
        max_courses_for_req = int(needed_condition.split("<")[1]) - 1
    elif ">=" in needed_condition:
        max_courses_for_req = float('inf')  # actually limited by total program credits/other reqs
    elif ">" in needed_condition:
        max_courses_for_req = float('inf')  # actually limited by total program credits/other reqs

    def generate_combinations_for_requirement(candidate_index, courses_taken_for_req, current_program_courses):
        if courses_taken_for_req > max_courses_for_req:  # Stop if we've taken too many for this requirement
            return

        if candidate_index == len(course_options):  # Reached end of candidates for this requirement
            if check_satisfied(needed_condition, courses_taken_for_req):  # Check if we satisfied the needed condition
                generate_programs(requirement_index + 1, current_program_courses)  # Move to next requirement
            return

        # Option A: Don't take the current candidate course
        generate_combinations_for_requirement(candidate_index + 1, courses_taken_for_req, current_program_courses)

        # Option B: Take the current candidate course if no conflict
        candidate_course_code = course_options[candidate_index]
        candidate_course = courses[candidate_course_code]
        # Construct current program schedule for conflict check
        current_program_schedule_for_check = {
            "schedule": []}
        for course_code in current_program_courses:
            current_program_schedule_for_check["schedule"].extend(courses[course_code]['schedule'])

        if not check_program_course_conflict(current_program_schedule_for_check, candidate_course['schedule']):
            updated_program_courses = current_program_courses.copy()
            #updated_program_courses[candidate_course_code] = candidate_course
            updated_program_courses.append(candidate_course_code)
            generate_combinations_for_requirement(candidate_index + 1, courses_taken_for_req + 1,
                                                  updated_program_courses)

    generate_combinations_for_requirement(0, 0, current_program_courses.copy())


print("Generating possible programs...")
generate_programs(0, [])
print(f"Found {len(possible_programs)} possible programs:")

In [ ]:



limit_results = 5
filter_func = lambda program: program['total_days'] == 3
sort_func = None  #lambda program: program['total_days']
print_wanted = False
return_wanted = True
save_txt = "test.txt"
include_schedule = True

fetched_programs = list_programs(possible_programs, filter_func=filter_func, sort_func=sort_func,
                                 print_wanted=print_wanted,
                                 return_wanted=return_wanted, save_txt=save_txt, include_schedule=include_schedule,
                                 limit_results=limit_results)